## **Notebook Content**

Project: *Customer Prioritization Under Constraints*

File: 01_data_cleaning.ipynb <br>
Author: Bryan Melvida

Purpose:
- Ingest raw transactional data
- Assess data quality, consistency, and anomalies
- Apply targeted data handling and corrections to prepare data for feature engineering and modeling

Input: [`Online Retail.xlsx`](../data/raw/Online%20Retail.xlsx) <br>
Related Documentation: [`customer_raw_dictionary.md`](../docs/raw/customer_raw_dictionary.md)

Output: [`customer_cleaned.parquet`](../data/cleaned/customer_cleaned.parquet)<br>
Related Documentation: [`data_handling_log.md`](../docs/data_handling/data_handling_log.md)

<br>

---

<br>

In [2]:
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
from pathlib import Path

import sys
sys.path.append('../')
import src.assessment_views as av

import pandas as pd
import numpy as np

**Load Dataset**

In [3]:
# Prevent reading xlsx if parquet file format exists
PARQUET_PATH = Path("../data/raw/customer_raw.parquet")
XLSX_PATH = Path("../data/raw/Online Retail.xlsx")

if PARQUET_PATH.exists() and PARQUET_PATH.stat().st_size > 0:
    df = pd.read_parquet(PARQUET_PATH)
    loaded_from = "parquet"
else:
    df = pd.read_excel(XLSX_PATH)
    loaded_from = "xlsx"

print(f"Dataset loaded from: {loaded_from}")

Dataset loaded from: parquet


**Export to Parquet**

In [4]:
if loaded_from == "xlsx":
    df["InvoiceNo"] = df["InvoiceNo"].astype(str)
    df["StockCode"] = df["StockCode"].astype(str)
    df["Description"] = df["Description"].astype(str)

    df.to_parquet(PARQUET_PATH, engine="pyarrow")

<br>

---

## **Data Assessment**
Evaluate overall data readiness to surface structural issues and risks before analysis proceeds

**Data Profiling & Structure Audit**

In [5]:
av.df_summary(df)

Total Rows: 541,909
Total Columns: 8
Total Null Values: 135,080

                  data type  # unique  # non-null  # null     % null
InvoiceNo            object     25900      541909       0   0.000000
StockCode            object      4070      541909       0   0.000000
Description          object      4224      541909       0   0.000000
Quantity              int64       722      541909       0   0.000000
InvoiceDate  datetime64[ns]     23260      541909       0   0.000000
UnitPrice           float64      1630      541909       0   0.000000
CustomerID          float64      4372      406829  135080  24.926694
Country              object        38      541909       0   0.000000


<br>

**Continuous Variable Validation**

In [6]:
df.describe(include= ['number']).T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


<br>

**Duplicate Check**

In [7]:
duplicates = df.loc[df.duplicated(keep=False), :]

print(f'Total Duplicated Records: {len(duplicates):,}')
print(f'Unique InvoiceNo Duplicated Records: {len(set(duplicates["InvoiceNo"])):,}')
print(f'Unique CustomerID Duplicate Records: {len(set(duplicates["CustomerID"])):,}')

Total Duplicated Records: 10,147
Unique InvoiceNo Duplicated Records: 1,933
Unique CustomerID Duplicate Records: 1,045


<br>

**Cancelled Invoices Check**

Reference: <br>
`InvoiceNo` data dictionary states that codes starting with letter "C" indicate cancelled transactions

In [8]:
cancelled_invoices = df.loc[df['InvoiceNo'].str.startswith('C'), :]
cancelled_invoices.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [9]:
cancelled_invoices.describe(include= ['number']).T

,count,mean,std,min,25%,50%,75%,max
Quantity,9288.0,-29.885228,1145.786965,-80995.00,-6.00,-2.00,-1.00,-1.0
UnitPrice,9288.0,48.393661,666.600430,0.01,1.45,2.95,5.95,38970.0
CustomerID,8905.0,14991.667266,1706.772357,12346.00,13510.00,14895.00,16393.00,18282.0


In [10]:
n_unique_cancelled_invoice = cancelled_invoices['InvoiceNo'].nunique()
n_unique_invoice = df['InvoiceNo'].nunique()
cancelled_invoice_pct = (n_unique_cancelled_invoice/ n_unique_invoice) * 100

print(f'Cancelled Invoice Count: {n_unique_cancelled_invoice:,} ({(cancelled_invoice_pct):.2f}%)',
      f'out of {n_unique_invoice:,}')

Cancelled Invoice Count: 3,836 (14.81%) out of 25,900


<br>

**Field Relationship Validation**

Reference:<br>
The data dictionary states that `StockCode` and `Description` are expected to have a one-to-one mapping relationship

In [11]:
unique_code_desc = df.groupby('StockCode')['Description'].nunique()
incosistent_stockcode = unique_code_desc[unique_code_desc > 1].index

print(f'Total Inconsistent StockCodes: {len(incosistent_stockcode)}')
print(f'Mapped `Descriptions` from inconsistent StockCode: {incosistent_stockcode[0]}')

set(df.loc[df['StockCode'] == incosistent_stockcode[0], 'Description'])

Total Inconsistent StockCodes: 1324
Mapped `Descriptions` from inconsistent StockCode: 10002


{'INFLATABLE POLITICAL GLOBE ', 'nan'}

<br>

Assess literal `"nan"` values presence across columns

In [12]:
nan_str_dict = {}

for column in df.columns:
    nan_str_dict[column] = len(df[column][df[column] == 'nan'])

nan_str_dict

{'InvoiceNo': 0,
 'StockCode': 0,
 'Description': 1454,
 'Quantity': 0,
 'InvoiceDate': 0,
 'UnitPrice': 0,
 'CustomerID': 0,
 'Country': 0}

<br>

Assess whether observed `StockCode` inconsistencies are associated with literal `"nan"` values

In [13]:
clean_unique_code_desc = df.loc[df['Description'] != 'nan'].groupby('StockCode')['Description'].nunique()
clean_inconsistent_stockcode = clean_unique_code_desc[clean_unique_code_desc > 1].index

print(f'Mapped `Description` from inconsistent StockCode: {clean_inconsistent_stockcode[0]}')

set(df.loc[(df['StockCode'] == clean_inconsistent_stockcode[0]) & 
           (df['Description'] != 'nan'),
           'Description'])

Mapped `Description` from inconsistent StockCode: 10080


{'GROOVY CACTUS INFLATABLE', 'check'}

<br>

**Identifier Format Validation**

Reference:<br>
- The data dictionary specifies `StockCode` as a 5-digit unique identifier
- `CustomerID` is excluded from format validation based on continuous variable validation results

In [14]:
unique_stockcodes = df['StockCode'].astype(str).drop_duplicates().to_frame(name='StockCode')

unique_stockcodes['digit_count'] = (
    unique_stockcodes['StockCode'].apply(lambda code: sum(char.isdigit() for char in code))
)

irregular_codes = unique_stockcodes.loc[unique_stockcodes['digit_count'] != 5, 'StockCode']

print("Irregular StockCodes Samples")
print("-" * 28)

for code in irregular_codes[:5]:
    print(code)

Irregular StockCodes Samples
----------------------------
POST
D
C2
DOT
M


<br>

---

### **Assessment Findings**
Consolidated summary of data quality issues identified during the assessment

**Missing Values**
- `CustomerID` contains 135,080 null values, representing 24.9% of all records.

**Duplicate Records**
- 10,147 duplicate rows identified.
    - Associated with 1,933 unique `InvoiceNo`
    - Involving 1,045 unique `CustomerID`

**Cancelled Transactions**
- 3,836 cancelled invoices, accounting for 14.81% of total invoices.

**Field Relationship Inconsistencies**
- `StockCode` does not consistently map to a single `Description`.
- Literal string value `"nan"` is present in the Description field.

**Identifier Format Validation**
- `StockCode` includes non–5-digit identifier values, including alphabetic and alphanumeric codes.

**Extreme Value Ranges**
- `Quantity` and `UnitPrice` exhibit extreme positive and negative values that fall outside expected operational ranges

<br>

---

## **Data Handling & Correction**
Address identified data quality issues to produce a stable dataset suitable for downstream feature engineering and segmentation

**Missing Values**

In [15]:
df_correction = df.dropna(axis=0)
av.df_summary(df_correction)

Total Rows: 406,829
Total Columns: 8
Total Null Values: 0

                  data type  # unique  # non-null  # null  % null
InvoiceNo            object     22190      406829       0     0.0
StockCode            object      3684      406829       0     0.0
Description          object      3896      406829       0     0.0
Quantity              int64       436      406829       0     0.0
InvoiceDate  datetime64[ns]     20460      406829       0     0.0
UnitPrice           float64       620      406829       0     0.0
CustomerID          float64      4372      406829       0     0.0
Country              object        37      406829       0     0.0


<br>

**Duplicate Records**

In [16]:
df_correction.loc[df_correction.duplicated(keep= False), :].sort_values(
    ['InvoiceNo', 'StockCode','InvoiceDate','CustomerID']
    ).head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
521,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
565,536412,21448,12 DAISY PEGS IN WOOD BOX,2,2010-12-01 11:49:00,1.65,17920.0,United Kingdom
578,536412,21448,12 DAISY PEGS IN WOOD BOX,1,2010-12-01 11:49:00,1.65,17920.0,United Kingdom


In [17]:
df_correction_v2 = df_correction.drop_duplicates(keep='first')
av.df_shape(df_correction_v2)

print(f"Total Duplicated Records: {df_correction_v2.duplicated().sum()}")

Total Rows: 401,604
Total Columns: 8
Total Duplicated Records: 0


<br>

**Field Relationship Inconsistencies**

In [18]:
df_correction_v2.loc[df_correction_v2['Description'] == 'nan']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [19]:
df_correction_v3 = df_correction_v2.loc[~df_correction_v2['StockCode'].isin(clean_inconsistent_stockcode)]

code_to_desc_max_count = df_correction_v3.groupby('StockCode')['Description'].nunique().max()

print(f"StockCode per Description: {code_to_desc_max_count}")
print(f"Unique Count StockCode: {df_correction_v3['StockCode'].nunique()}")
print(f"Unqiue Count Description: {df_correction_v3['Description'].nunique()}")

StockCode per Description: 1
Unique Count StockCode: 3046
Unqiue Count Description: 3033


In [20]:
desc_to_code = df_correction_v3.groupby('Description')['StockCode'].nunique()
desc_to_code.loc[desc_to_code > 1].head()

Description
COLUMBIAN CANDLE RECTANGLE         2
COLUMBIAN CANDLE ROUND             2
EAU DE NILE JEWELLED PHOTOFRAME    2
FRENCH LATTICE CUSHION COVER       2
FROSTED WHITE BASE                 2
Name: StockCode, dtype: int64

In [21]:
inconsistent_description = desc_to_code.loc[desc_to_code > 1].index

df_correction_v4 = df_correction_v3.loc[~df_correction_v3['Description'].isin(inconsistent_description)]

desc_to_code_max_count = df_correction_v3.groupby('StockCode')['Description'].nunique().max()

print(f"Description per StockCode: {desc_to_code_max_count}")
print(f"Unique Count StockCode: {df_correction_v4['StockCode'].nunique()}")
print(f"Unqiue Count Description: {df_correction_v4['Description'].nunique()}")

print()
av.df_shape(df_correction_v4)

Description per StockCode: 1
Unique Count StockCode: 3020
Unqiue Count Description: 3020

Total Rows: 313,183
Total Columns: 8


<br>

**Identifier Format Validation**

In [22]:
df_correction_v5 = df_correction_v4.loc[~df_correction_v3['StockCode'].isin(irregular_codes)]

standard_unique_stockcodes = (
    df_correction_v5['StockCode'].astype(str)
    .drop_duplicates()
    .to_frame(name='StockCode')
)

standard_unique_stockcodes['digit_count'] = (
    standard_unique_stockcodes['StockCode'].apply(lambda code: sum(char.isdigit() for char in code))
)

non_standard_count = standard_unique_stockcodes.loc[
    standard_unique_stockcodes['digit_count'] != 5, 'StockCode'].sum()

print(f'Non-standard StockCode Count: {non_standard_count}')
av.df_shape(df_correction_v5)


Non-standard StockCode Count: 0
Total Rows: 311,268
Total Columns: 8


<br>

**Cancelled Transactions & Extreme Value Ranges**

>- Cancelled transactions explain the observed extreme ranges in `Quantity` and `UnitPrice`. 
>- These records are retained, as they represent meaningful customer behavior and are required for downstream behavioral feature generation

In [23]:
# Export cleaned dataset
df_correction_v4.to_parquet('../data/cleaned/customer_cleaned.parquet', engine= 'pyarrow', index= False)

<br>

---

### **Handling & Correction Outcome**
The following summarizes the final actions taken during data cleaning and the resulting characteristics of the analysis dataset


><ins>Missing Values</ins>
>- Remove records with missing `CustomerID` 
>
><ins>Duplicate Records</ins>
>- Remove duplicate records
>
><ins>Field Relationship Inconsistencies</ins>
>- Remove records where a single `StockCode` maps to multiple `Description` values
>- Remove records with literal "nan" values in Description
>
><ins>Identifier Format Validation</ins>
>- Remove records where `StockCode` does not conform to the 5-digit format
>
><ins>Cancelled Transactions</ins>
>- Retain cancelled transactions for feature engineering as derived behavior variables
>
><ins>Extreme Value Ranges</ins>
>- Retain extreme `Quantity` and `UnitPrice` values associated with cancelled transactions for derived behavioral features

<br>

---

## **End of Notebook**
- Data has been cleaned, validated, and is ready for feature engineering.
- Key data issues identified and resolutions are documented in: [`data_handling_log.md`](../docs/data_handling/data_handling_log.md)
- Ready for downstream notebook: [`02_feature_engineering.ipynb`](../notebooks/02_feature_engineering.ipynb)